In [2]:
import numpy as np
import pandas as pd
import geocoder 
import time

Let's obtain the dataframe as done in the previous notebook/question

In [3]:
to_scrape_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(to_scrape_url)[0]
df['Borough'] = df['Borough'].replace('Not assigned', np.nan)
df = df.dropna().reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
lats = np.zeros(len(df))
longs = np.zeros(len(df))
# get latitude and longitude for each postal code
for i in range(len(df)):
    lat_lng_coords = None
    postal_code = df.iloc[i, 0]
    print(i, postal_code)
    k = 0
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        time.sleep(0.01) # add a little delay not to waste calls
        k += 1
        if(k > 50):
            print('Data collection failed')
            break
    lats[i] = lat_lng_coords[0]
    longs[i] = lat_lng_coords[1]
# Add lat and long to dataframe
df['Latitude'] = lats
df['Longitude'] = longs
df.head()

0 M3A
Data collection failed


TypeError: 'NoneType' object is not subscriptable

Since geocode library does not appear to work I will load in the location data from a csv file. and merge it to my dataframe after making sure they are in the same order

In [13]:
loc_data_url = 'http://cocl.us/Geospatial_data'
loc_data = pd.read_csv(loc_data_url)
df = df.sort_values('Postal Code').reset_index(drop=True)
loc_data = loc_data.sort_values('Postal Code').reset_index(drop=True)
print('Do the two datframes correspond? {}'.format((df['Postal Code'] == loc_data['Postal Code']).all()))

Do the two datframes correspond? True


Do the two datframes correspond ? True
